# Agent, RL and MultiEnvironment
Try this notebook out interactively with: [![Binder](./img/badge_logo.svg)](https://mybinder.org/v2/gh/rte-france/Grid2Op/master)

It is recommended to have a look at the [00_Introduction](00_Introduction.ipynb), [01_Grid2opFramework](01_Grid2opFramework.ipynb) and [03_Action](03_Action.ipynb) and especially [04_TrainingAnAgent](04_TrainingAnAgent.ipynb) notebooks before getting into this one.

**Objectives**

In this notebook we will introduce :
* what is a "SingleEnvMultiProcess" (previously [MultiEnvironment](https://grid2op.readthedocs.io/en/v0.9.4/environment.html#grid2op.Environment.MultiEnvironment))
* how can it be used with an agent
* how can it be used to train a agent that uses different environments (using the [l2rpn-baselines](https://l2rpn-baselines.readthedocs.io/en/master/) python package)

<img src="https://colab.research.google.com/assets/colab-badge.svg" width="200">
Execute the cell below by removing the # character if you use google colab !

Cell will look like:
```python
!pip install grid2op[optional]  # for use with google colab (grid2Op is not installed by default)
```
<img src="https://colab.research.google.com/assets/colab-badge.svg" width="200">

In [ ]:
# !pip install grid2op[optional]  # for use with google colab (grid2Op is not installed by default)

In [10]:
res = None
try:
    from jyquickhelper import add_notebook_menu
    res = add_notebook_menu()
except ModuleNotFoundError:
    print("Impossible to automatically add a menu / table of content to this notebook.\nYou can download \"jyquickhelper\" package with: \n\"pip install jyquickhelper\"")
res

In [2]:
import grid2op
TRAINING_STEP = 100

c:\Users\thoma\anaconda3\envs\MA\Lib\site-packages\grid2op\Backend\pandaPowerBackend.py:32: UserWarning: Numba cannot be loaded. You will gain possibly massive speed if installing it by 
	c:\Users\thoma\anaconda3\envs\MA\python.exe -m pip install numba

  warnings.warn(


## I) Make a regular environment and agent

By default we use the test environment. But by passing `test=False` in the following function will automatically download approximately 300MB from the internet and give you 1000 chronics instead of 2 used for this example.

In [11]:
env = grid2op.make("l2rpn_case14_sandbox", test=True)

c:\Users\thoma\anaconda3\envs\MA\Lib\site-packages\grid2op\MakeEnv\Make.py:438: UserWarning: You are using a development environment. This environment is not intended for training agents. It might not be up to date and its primary use if for tests (hence the "test=True" you passed as argument). Use at your own risk.
  warnings.warn(_MAKE_DEV_ENV_WARN)


A lot of data have been made available for the default "rte_case14_realistic" environment. Including this data in the package is not convenient. 

We chose instead to release them and make them easily available with a utility. To download them in the default directory ("~/data_grid2op/case14_redisp")  just pass the argument "test=False" (or don't pass anything else) as local=False is the default value. It will download approximately 300Mo of data.

## II) Train a standard RL Agent

Make sure you are using a computer with at least 4 cores if you want to notice some speed-ups.

In [12]:
from grid2op.Environment import SingleEnvMultiProcess
from grid2op.Agent import DoNothingAgent
NUM_CORE = 2

### IIIa) Using the standard open AI gym loop

Here we demonstrate how to use the multi environment class. First let's create a multi environment.

In [5]:
# create a simple agent
agent = DoNothingAgent(env.action_space)

# create the multi environment class
multi_envs = SingleEnvMultiProcess(env=env, nb_env=NUM_CORE)

A multi environment is just like using a regular environment but instead of dealing with one action, and one observation, it requires to be sent multiple actions, and returns a list of observations as well. 

It requires a grid2op environment to be initialized and creates some specific "workers", each is a replication of the initial environment. None of the "worker"s can be accessed directly. Supported methods are:
- multi_env.reset
- multi_env.step
- multi_env.close

These methods have similar behaviour as "env.step", "env.close" or "env.reset".


It can be used the following way:

In [6]:
# initiliaze some variable with the proper dimension
obss = multi_envs.reset()
rews = [env.reward_range[0] for i in range(NUM_CORE)]
dones = [False for i in range(NUM_CORE)]
obss

array([<grid2op.Space.GridObjects.CompleteObservation_l2rpn_case14_sandbox object at 0x000001AE462987A0>,
      dtype=object)

In [7]:
dones

[False, False]

As you can see, obs is not a single obervation, but a list (numpy nd array to be precise) of 4 observations (if 4 cores were used, otherwise the length of the array is the number of cores), each one being an observation of a given "worker" environment.

Worker environments are always called in the same order. It means the first observation of this vector will always correspond to the first worker environment. 


Similarly to Observation, the "step" function of a multi_environment takes as input a list of multiple actions, each action will be implemented in its own environment. It returns a list of observations, a list of rewards, and boolean list of whether or not the worker environment suffers from a game over (in that case this worker environment is automatically restarted using the "reset" method.)

Because orker environments are always called in the same order, the first action sent to the "multi_env.step" function will also be applied on this first environment.

It is possible to use it as follow:

In [8]:
# initialize the vector of actions that will be processed by each worker environment.
acts = [None for _ in range(NUM_CORE)]
for env_act_id in range(NUM_CORE):
    acts[env_act_id] = agent.act(obss[env_act_id], rews[env_act_id], dones[env_act_id])
    
# feed them to the multi_env
obss, rews, dones, infos = multi_envs.step(acts)

# as explained, this is a vector of Observation (as many as NUM_CORE in this example)
obss

array([<grid2op.Space.GridObjects.CompleteObservation_l2rpn_case14_sandbox object at 0x000001AE49C289E0>,
      dtype=object)

The multi environment loop is really close to the "gym" loop:

In [9]:
# performs the appropriated steps
for i in range(10):
    acts = [None for _ in range(NUM_CORE)]
    for env_act_id in range(NUM_CORE):
        acts[env_act_id] = agent.act(obss[env_act_id], rews[env_act_id], dones[env_act_id])
    obss, rews, dones, infos = multi_envs.step(acts)

    # DO SOMETHING WITH THE AGENT IF YOU WANT
    ## agent.train(obss, rews, dones)
    

# close the environments created by the multi_env
multi_envs.close()

On the above example, `TRAINING_STEP` steps are performed on `NUM_CORE` environments in parrallel. The agent has then acted `TRAINING_STEP * NUM_CORE` (=`10 * 4 = 40` by default) times on `NUM_CORE` different environments.

### III.b) Practical example (work in progress)

We reuse the code of the Notebook [04_TrainingAnAgent](04_TrainingAnAgent.ipynb) to train a new agent we strongly recommend you to have a look at it if it is not done already.

In this notebook, we focus on how to make your agent interact with multiple environments at the same time (**eg** it means that the batch of data the agent receives comes from different instance of the same environment, this is a technique used in [Asynchronous Actor Critic - A3C](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-8-asynchronous-actor-critic-agents-a3c-c88f72a5e9f2) type of models for example).

You will see with this method you can learn the same baseline with minimal (actually without any) changes in the above way.

#### Note on the implementation
The most common code for A3C agent can be summarized in the image below (image credit [this blog post](https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-8-asynchronous-actor-critic-agents-a3c-c88f72a5e9f2)):
![](https://miro.medium.com/max/1400/1*YtnGhtSAMnnHSL8PvS7t_w.png)

In this image you see different version of the agent that interacts with different versions of the (same) environment (represented here in the different *workers*). And, from time to time, each "worker" will send its weights to the "global network" and an update procedure will be run there. Though this framework can perfectly be implemented in grid2op, the *SingleEnvMultiProcess* class works a bit differently.

Actually, in this *SingleEnvMultiProcess* class, there exists only one copy of the "global network" that is kept into the main "thread" (which to be precise is a process) and that interacts (send actions to and receives observations from) with different independant environments. It is really similar to the [SubprocVecEnv](https://github.com/openai/baselines/blob/ea25b9e8b234e6ee1bca43083f8f3cf974143998/baselines/common/vec_env/subproc_vec_env.py#L39) of open ai gym.

This is especially suited in the case of powersystem operations, as it can be quite computationnally expensive to solve for the powerflow equations at each nodes of the grid (also called [Kirchhoff's laws](https://en.wikipedia.org/wiki/Kirchhoff%27s_circuit_laws))

Here, you will see in the next cell how to (*not really*) change it to train a agent on different environments: